블랙잭 환경을 테스트하는 예제입니다. 교재에 관련 부분은 없습니다.

`import numpy as np` 대신 노트북 안에서는 `%pylab`을 사용해도 됩니다. `%pylab`은 아래 코드와 같은 역할을 합니다.

```python
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *
```

`sys.path`는 불러올 외부 코드를 찾는 위치입니다.

```
reinforcement-learning/ -+- MC/ -+- Blackjack Playground.ipynb
                         |
                         +- lib/ -+- envs/ -+- blackjack.py (class BlackjackEnv)
                         |        |
                         |        +- ...
                         +- ...
```

이 파일(`Blackjack Playground.ipynb`)이 `blackjack.py` 코드를 불러오려면, 먼저 외부 코드를 찾기 시작할 위치를 한 디렉토리 위로 올려야 합니다(`sys.path.append("../")`). 그 다음 `lib`과 `envs` 디렉토리 안에 있는 `blackjack.py`를 불러옵니다. 여기서 마지막 `.py`는 쓰지 않습니다. `blackjack.py` 파일 안에 우리가 사용할 `BlackjackEnv` 클래스가 있습니다. 그냥 `import lib.envs.blackjack`을 사용하면, 뒤 코드에서 `backjack.BlackjackEnv`로 환경 클래스를 지칭합니다. `from lib.envs.blackjack import BlackjackEnv`을 사용하면, 뒤 코드에서 `BlackjackEnv`로 환경 클래스를 지칭할 수 있습니다. 여기서는 `blackjack.py` 파일 안에 `BlackjackEnv` 클래스만 있지만, 만약 다른 클래스도 있고 그 클래스도 사용하려면 `import` 뒤에 함께 적어둡니다.

원본과 달리 여기서는 `Blackjack Playground.ipynb`와 `blackjack.py` 파일이 같은 디렉토리 안에 두었기 때문에 `from blackjack import BlackjackEnv`를 사용해도 됩니다.

In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../")
#from lib.envs.blackjack import BlackjackEnv
from blackjack import BlackjackEnv

In [2]:
env = BlackjackEnv()

```python
def print_observation(observation):
```

observation은 MDP 상태입니다. 교재에서 정의한 블랙잭 게임의 상태는 (현재 선수가 가진 카드의 합, 딜러가 공개한 카드, 선수의 유저블 에이스 보유 여부)입니다. 화면에 출력할 때 `printf` 스타일로 `"Player Score: %d (Usable Ace: %s), Dealer Score: %d" % (score, usable_ace, dealer_score)`를 사용해도 됩니다.

```python
def strategy(observation):
```

상태에 대한 행동을 결정하는 정책 함수입니다. 여기서는 합이 20 보다 같거나 크면 멈추고, 작으면 계속하는 단순한 정책을 사용합니다. 멈추기(stick) 행동은 0, 계속(hit) 행동은 1입니다. 행동 값으로 숫자 0과 1을 사용하더라도, 0과 1을 코드에서 사람이 이해하기 쉽게 정의해 두면 좋습니다. (예, `STICK = 0`)

`return 0 if score >= 20 else 1`은 아래 문장과 동일합니다. 차이는 C 같은 언어의 `score >= 20 ? 0 : 1` 처럼 문장이 아니라 표현식입니다.

```
if score >= 20:
    return 0
else:
    return 1
```

에피소드를 20회 반복하며 게임 진행 상황을 출력합니다. 환경을 초기화하고(`env.reset`), 에피소드가 끝날 때까지(`if done:`) 충분한 횟수만큼 행동을 반복합니다. 제일 낮은 숫자인 1 카드를 21번 뽑으면 게임이 끝나기 때문에 여기서 사용한 100회 이전에 반드시 게임이 끝납니다. 100회 반복 대신 무한 반복해도 (`while True`:) 무방합니다. OpenAI Gym 환경의 `step()` 함수는 행동을 받아서 (관찰, 보상, 종료여부, 기타정보)를 반환합니다.

In [3]:
def print_observation(observation):
    score, dealer_score, usable_ace = observation
    print("Player Score: {} (Usable Ace: {}), Dealer Score: {}".format(
          score, usable_ace, dealer_score))

def strategy(observation):
    score, dealer_score, usable_ace = observation
    # Stick (action 0) if the score is > 20, hit (action 1) otherwise
    return 0 if score >= 20 else 1

for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        print_observation(observation)
        action = strategy(observation)
        print("Taking action: {}".format( ["Stick", "Hit"][action]))
        observation, reward, done, _ = env.step(action)
        if done:
            print_observation(observation)
            print("Game end. Reward: {}\n".format(float(reward)))
            break

Player Score: 20 (Usable Ace: False), Dealer Score: 1
Taking action: Stick
Player Score: 20 (Usable Ace: False), Dealer Score: 1
Game end. Reward: -1.0

Player Score: 20 (Usable Ace: False), Dealer Score: 1
Taking action: Stick
Player Score: 20 (Usable Ace: False), Dealer Score: 1
Game end. Reward: 1.0

Player Score: 20 (Usable Ace: False), Dealer Score: 5
Taking action: Stick
Player Score: 20 (Usable Ace: False), Dealer Score: 5
Game end. Reward: 1.0

Player Score: 12 (Usable Ace: False), Dealer Score: 2
Taking action: Hit
Player Score: 21 (Usable Ace: False), Dealer Score: 2
Taking action: Stick
Player Score: 21 (Usable Ace: False), Dealer Score: 2
Game end. Reward: 1.0

Player Score: 12 (Usable Ace: False), Dealer Score: 9
Taking action: Hit
Player Score: 18 (Usable Ace: False), Dealer Score: 9
Taking action: Hit
Player Score: 23 (Usable Ace: False), Dealer Score: 9
Game end. Reward: -1.0

Player Score: 18 (Usable Ace: False), Dealer Score: 6
Taking action: Hit
Player Score: 24 (Usa